# Naver 영화 네티즌 평점 수집 - 작은 아씨들, 2019

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=186613'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

# iframe의 src 주소값을 얻어야 함

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=186613&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [4]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

# 읽을 페이지 수 계산

In [5]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>3,387</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [6]:
count = count.find('em').string     # get_text() 대신에 string
count

'3,387'

In [7]:
count = int(count.replace(',',''))
count

3387

In [8]:
import math
total_page = math.ceil(count/10)
total_page

339

# 전체 리뷰에서 데이터 수집하기

In [10]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [13]:
import pandas as pd
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df.head(10)

,평점,일시,감상평
0,10,2020.02.12 20:55,왜 여성은 사랑을 해야하냐며 말하면서도 사랑을 받고 싶다고 외치던 장면이 너무나 울...
1,10,2020.02.12 09:02,재밌다고 느끼면 추천을 아니다 비추천
2,8,2020.02.12 13:12,철없는 막내로만 재현되어왔던 에이미 캐릭터의 재발견. 어쩌면 자매 중에서 제일 고민...
3,10,2020.02.12 13:27,“자매를 미워하기엔 인생이 너무 짧아.“ 라는 명대사가 내 마음 속에 강하게 들어왔다.
4,10,2020.02.12 21:18,조가 여자의 전부가 사랑이라는 말이 지긋지긋하지만 너무 외롭다는 말이 공감이 간다....
5,10,2020.02.12 11:58,영화를 보고 나니 조를 응원하는 내 자신을 발견했다. 잔잔한 감동과 재미가 있는 영화.
6,9,2020.02.12 13:19,'우리의 어린시절이 끝이났다니 믿겨지지 않아.'격렬하고도 잔잔한 따뜻했던 가족이야기.
7,10,2020.02.12 14:51,13살 때 1995년도판 &lt;작은 아씨들&gt;을 보고 에이미를 네 자매 중 가...
8,10,2020.02.12 17:10,"엠마왓슨 진짜 존예.. 영화 너무 사랑스럽고 이뻐요..,"
9,10,2020.02.12 21:04,"영화보다가 나도 모르게 계속 눈물이 나더라.조에 입혀진 나를 만나고, 아직 갈팡질팡..."


In [14]:
df.to_csv('Little Women.csv')

In [15]:
len(df)

3387

In [ ]:
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=186613&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1